## Import Libraries

In [8]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate
from scipy.spatial import distance
import warnings
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

## Initialize Parameters 

In [13]:
case  = 'wat'

if case=='atm':
    theta  = 0
    n      = 1.00029
    c      = 299792458/n
    h      = 25000
    L      = 100
    v      = n * c
    times  = linspace(1e-6,1e-4,300000,endpoint=True)
    
elif case=='wat':
    n      = 1.33
    c      = 299792458/n
    h      = 4.5
    D      = 7.3   #--------------------- diameter of tank
    R      = 7.3/2 #--------------------- radius of tank
    v      = n * c
    times  = linspace(1e-10,1e-7,200000)
    
    c1     = (0, 0, 0)  #------------------------------------ Central    PMT number 1 
    c2     = (1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0) #------ Non-Radial PMT number 2
    c3     = (1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0) #------ Non-Radial PMT number 3
    c4     = (1.85*cos(0     ) , 1.85*sin(0     ), 0) #------ Radial     PMT number 4
    x1,y1  = (1.2,1.2) #----------------------------------------- Coordinates of particle at top tanker lid      #------PARAM
    theta  = 165   #----------------------------------------- Zenith  varies from 90 to 180                    #------PARAM
    phi    =  10   #----------------------------------------- Azimuth varies from  0 to 360                    #------PARAM
    theta  = deg2rad(theta) #-------------------------------- Convert zenith to radians
    phi    = deg2rad(phi  ) #-------------------------------- Convert azimuth to radians

    x2     = x1 + h * tan(theta) * cos(phi) # ------------------- Calculate bottom x coordinate
    y2     = y1 + h * tan(theta) * sin(phi) #-------------------- Calculate bottom y coordinate
    cp     = (x2,y2,0)
    print('A = {}'.format((x1,y1,h)))
    print('B = {}'.format(cp))

    d1     = distance.euclidean(cp,c1)  #------------------------ Calculate the distances of the point 
    d2     = distance.euclidean(cp,c2)  #------------------------ where the particle hits the bottom of 
    d3     = distance.euclidean(cp,c3)  #------------------------ the tank from the 4 PMTs
    d4     = distance.euclidean(cp,c4)
    theta2    = pi - theta
    del theta
    theta     = theta2
    print('Particle distances from PMTs    : {} m,   {} m,   {} m,   {} m'.format( round(d1,3),round(d2,3),round(d3,3),round(d4,3)))
    print(theta)
    cbeta = v/c
    cang  = arccos(1/(n*cbeta))
    print("Cherenkov Angle:", rad2deg(cang))

A = (1.2, 1.2, 4.5)
B = (0.01254701046211526, 0.9906199996215443, 0)
Particle distances from PMTs    : 0.991 m,   1.119 m,   2.757 m,   2.087 m
0.2617993877991496
Cherenkov Angle: 55.57527028287004


## Computations 

In [14]:
def entry_brightness(h,c,v,theta,L):
    z        = h/cos(theta)
    k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
    t_shower = 0
    t_light  = k/c
    t_total  = t_shower + t_light
    tt       = t_total
    den      = (c**2-v**2)
    num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
    num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta)  \
               + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 \
               - 2*c*c*h*L*tan(theta)))
    zp       = ((num1_z - num2_z) / den)
    v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
    v_B      = num2_z
    vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
    kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
    alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
    vtp      = vp * sin(alphap)
    omegap   = (vtp / kp)
    bp       = abs(omegap / (kp**2))
    return bp

#==========================================================================================================================================
#==========================================================================================================================================

def plus_b_vs_phi(phip,bp,color,mylabel,phi):
    plt.plot(phip, bp, c=color, ls='-', label=mylabel)
    plt.axhline(1,  c='k', ls=':')
    plt.axvline(phi,c='k', ls=':')
    plt.text(phi+1, 0.5, r'$\theta_C$ = {}'.format(round(phi,2))     , fontsize=18)
    plt.xlabel(r'angular locations $\phi_{C}\;(in\;degrees)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
def minus_b_vs_phi(phim,bm,color,mylabel,phi):
    plt.plot(phim, bm, c=color, ls='--',label=mylabel)
    plt.axhline(1,  c='k', ls=':')
    plt.axvline(phi,c='k', ls=':')
    plt.text(phi+1, 0.5, r'$\theta_C$ = {}'.format(round(phi,2))     , fontsize=18)
    plt.xlabel(r'angular locations $\phi_{C}\;(in\;degrees)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
def both_b_vs_phi(phip,bp,phim,bm,color,mylabel,phi):
    plt.plot(phip, bp, c=color, ls='-', label=mylabel)
    plt.plot(phim, bm, c=color, ls='--')
    plt.axhline(1,  c='k', ls=':')
    plt.axvline(phi,c='k', ls=':')
    plt.text(phi+1, 0.5, r'$\theta_C$ = {}'.format(round(phi,2))     , fontsize=18)
    plt.xlabel(r'angular locations $\phi_{C}\;(in\;degrees)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    
#==========================================================================================================================================
#==========================================================================================================================================
    
def plus_b_vs_z(zp,bp,color,mylabel,zc):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(zp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    if (0<=zc<=h):
        plt.text(zc+0.03, 5000, r'$z_C$ = {}'.format(round(zc,2))     , fontsize=18)
        plt.axvline(zc_act,c='k',ls=':')
    plt.xlabel(r'image heights $z_{pm}\;(in\;meters)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    plt.xlim(0,h)
def minus_b_vs_z(zm,bm,color,mylabel,zc):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(zm, bm, c=color, ls='--',label=mylabel, lw=2.5)
    if (0<=zc<=h):
        plt.text(zc+0.03, 5000, r'$z_C$ = {}'.format(round(zc,2))     , fontsize=18)
        plt.axvline(zc_act,c='k',ls=':')
    plt.xlabel(r'image heights $z_{pm}\;(in\;meters)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    plt.xlim(0,h)
def both_b_vs_z(zp,bp,zm,bm,color,mylabel,zc):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(zp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.plot(zm, bm, c=color, ls='--', lw=2.5)
    if (0<=zc<=h):
        plt.text(zc+0.03, 5000, r'$z_C$ = {}'.format(round(zc,2))     , fontsize=18)
        plt.axvline(zc_act,c='k',ls=':')
    plt.xlabel(r'image heights $z_{pm}\;(in\;meters)$', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    plt.xlim(0,h)
    
#==========================================================================================================================================
#==========================================================================================================================================

def plus_b_vs_t(t,bp,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(t, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    plt.ylim(0,1e3)
def minus_b_vs_t(t,bm,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(t, bm, c=color, ls='--',label=mylabel, lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    plt.ylim(0,1e3)
def both_b_vs_t(tp,bp,tm,bm,color,mylabel):  # Use times  = linspace(1e-12,1e-7,2000000)
    plt.plot(tp, bp, c=color, ls='-', label=mylabel, lw=2.5)
    plt.plot(tm, bm, c=color, ls='--', lw=2.5)
    plt.axhline(1, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
    plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)        
    plt.yscale('log')
    plt.ylim(0,1e3)
    
#==========================================================================================================================================
#==========================================================================================================================================
    
def plus_phi_vs_t(tp,phip,color,mylabel,phi):
    plt.plot(tp, phip, c=color, ls='-' , label=mylabel)
    plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
    plt.axhline(phi, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.text(1.2, phi+1.5, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,80)
    plt.xlim(-0.1,5.5)
def minus_phi_vs_t(tm,phim,color,mylabel,phi):
    plt.plot(tm, phim, c=color, ls='--' , label=mylabel)
    plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
    plt.axhline(phi, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.text(1.2, phi+1.5, r'$\theta_C = %.2f ^{\circ}$'%phi , fontsize=15)
    plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,80)
    plt.xlim(-0.1,5.5)
def both_phi_vs_t(tp,phip,tm,phim,color,mylabel,phi):
    plt.plot(tp, phip, c=color, ls='-' , label=mylabel)
    plt.plot(tm, phim, c=color, ls='--')
    plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
    plt.axhline(phi, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.text(1.2, phi+1.5, r'$\theta_C = "%.2f" ^{\circ}$'%phi , fontsize=15)
    plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,80)
    plt.xlim(-0.1,5.5)
    
#==========================================================================================================================================
#==========================================================================================================================================
def plus_z_vs_t(tp,zp,color,mylabel,zc):
    plt.plot(tp, zp, c=color, ls='-' , label=mylabel, lw=2.5)
    plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
    if (0<=zc<=h):
        plt.text(2, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        plt.axhline(zc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.ylabel(r'image heights $z_{pm} (in\;meters)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,h)
def minus_z_vs_t(tm,zm,color,mylabel,zc):
    plt.plot(tm, zm, c=color, ls='--' , label=mylabel, lw=2.5)
    plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
    if (0<=zc<=h):
        plt.text(2, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        plt.axhline(zc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.ylabel(r'image heights $z_{pm} (in\;meters)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,h)
def both_z_vs_t(tp,zp,tm,zm,color,mylabel,zc):
    plt.plot(tp, zp, c=color, ls='-' , label=mylabel, lw=2.5)
    plt.plot(tm, zm, c=color, ls='--', lw=2.5)
    plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
    if (0<=zc<=h):
        plt.text(2, zc+0.05, r'$z_C = %.2f $'%(round(zc,2)) , fontsize=15)
        plt.axhline(zc, c='k', ls=':')
    plt.axvline(0, c='k', ls=':')
    plt.ylabel(r'image heights $z_{pm} (in\;meters)$'     , fontsize=18).set_rotation(90)
    plt.tick_params(axis='both', direction='in', labelsize=18)
    plt.ylim(0,h)

In [16]:
%matplotlib

plotme    = 'b vs z'
#plotme    = 'b vs phi'
#plotme    = 'b vs t'
#plotme    = 'b vs z'
#plotme = 'z vs t'



distances = [d1,d2,d3,d4]
colors    = ['k','r','b','g']
detectors = [1,2,3,4]
labels    = ['Central Detector', "Detector 2",'Detector 3','Detector 4']

for ds, color, detector, mylabel in zip(distances, colors, detectors, labels):
    
    print("==============================================")
    print("Processing detector %s"%detector)
    
    T, ZP, ZM, BP, BM, PHIP, PHIM = [], [], [], [], [], [], []
    L      = ds
    num_zc = -sqrt(-c**4 * L**2 * cos(theta)**2 + c**2 * L**2 * v**2 * cos(theta)**2) + c**2*L*sin(theta) - L*v**2*sin(theta)
    den    = (c**2-v**2)
    zc     = (num_zc/den)
    zc_act = zc * cos(theta)
    print('z_C : {} m'.format(round(zc_act,3)))

    for iii,t in enumerate(times):
        z        = h/cos(theta) - v*t
        k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
        t_shower = t
        t_light  = k/c
        t_total  = t_shower + t_light
        tt       = t_total

        num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
        num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
        zp       = ((num1_z - num2_z) / den)
        zm       = ((num1_z + num2_z) / den)
        v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
        v_B      = num2_z
        vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
        vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
        kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
        km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
        alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
        alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
        phip     = 90+theta-rad2deg(alphap)
        phim     = 90+theta-rad2deg(alpham)   
        vtp      = vp * sin(alphap)
        vtm      = vm * sin(alpham)
        omegap   = (vtp / kp)
        omegam   = (vtm / km)
        bp       = abs(omegap / (kp**2))
        bm       = abs(omegam / (km**2))
        
        ZM.append(zm*cos(theta))
        ZP.append(zp*cos(theta))
        BM.append(bm)            
        BP.append(bp)
        PHIP.append(phip)
        PHIM.append(phim)
        T.append(tt)
        
    K = sqrt( L**2 + zc**2 - 2*zc*L*cos(pi/2-theta))
    ALPHA   = (arccos( (zc*zc + K*K - L*L) / (2*zc*K) ))
    PHI     = 90+theta-rad2deg(ALPHA)

    ZP, ZM, T, BP, BM, PHIP, PHIM = array(ZP), array(ZM), array(T), array(BP), array(BM), array(PHIP), array(PHIM)
    Tn = T*1e+9 ; TT = (Tn-Tn.min())
    #hp = ZP
    #hm = ZM
    
    cond = ( ((ZP>=0) & (ZP<=h)) | ((ZM>=0) & (ZM<=h))) # Improve further if possible
    TT   = TT[cond] ; BM = BM[cond] ; BP = BP[cond] ; PHIP = PHIP[cond] ; PHIM = PHIM[cond] ; ZP=ZP[cond] ; ZM=ZM[cond]
    
    conp = [(ZP>=0) & (ZP<=h)]
    conm = [(ZM>=0) & (ZM<=h)]
    pluslen  = len (ZP[conp])
    minuslen = len (ZM[conm])
    print(pluslen,minuslen)
    if detector==1:
        den_bright = entry_brightness(h=h,c=c,v=v,theta=theta,L=L)
    
    if pluslen == 0:
        if minuslen == 0:
            print("Images outside tank. Skipping...")
        else:
            print('Only image moving down. Processing...')
            TT,ZM,BM,PHIM = TT[conm],ZM[conm],BM[conm]/den_bright,PHIM[conm]
            if plotme=='b vs phi':
                minus_b_vs_phi(phim=PHIM,bm=BM,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='b vs z':
                minus_b_vs_z(zm=ZM,bm=BM,color=color,mylabel=mylabel,zc=zc_act)
            elif plotme=='b vs t':
                minus_b_vs_t(t=TT,bm=BM,color=color,mylabel=mylabel)
            elif plotme=='phi vs t':
                minus_phi_vs_t(tm=TT,phim=PHIM,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='z vs t':
                minus_z_vs_t(tm=TT,zm=ZM,color=color,mylabel=mylabel,zc=zc_act)
    elif pluslen != 0:
        if minuslen == 0:
            print('Only image moving up. Processing...')
            TT,ZP,BP,PHIP = TT[conp],ZP[conp],BP[conp]/den_bright,PHIP[conp]
            if plotme=='b vs z':
                plus_b_vs_phi(phip=PHIP,bp=BP,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='b vs z':
                plus_b_vs_z(zp=ZP,bp=BP,color=color,mylabel=mylabel,zc=zc_act)
            elif plotme=='b vs t':
                plus_b_vs_t(t=TT,bp=BP,color=color,mylabel=mylabel)
            elif plotme=='phi vs t':
                plus_phi_vs_t(tp=TT,phip=PHIP,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='z vs t':
                plus_z_vs_t(tp=TT,zp=ZP,color=color,mylabel=mylabel,zc=zc_act)
        else:
            print('Both images moving. Processing...')
            TTm,ZM,BM,PHIM = TT[conm],ZM[conm],BM[conm]/den_bright,PHIM[conm]
            TTp,ZP,BP,PHIP = TT[conp],ZP[conp],BP[conp]/den_bright,PHIP[conp]
            if plotme=='b vs phi':
                both_b_vs_phi(phip=PHIP,bp=BP,phim=PHIM,bm=BM,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='b vs z':
                both_b_vs_z(zp=ZP,bp=BP,zm=ZM,bm=BM,color=color,mylabel=mylabel,zc=zc_act)
            elif plotme=='b vs t':
                both_b_vs_t(tp=TTp,bp=BP,tm=TTm,bm=BM,color=color,mylabel=mylabel)
            elif plotme=='phi vs t':
                both_phi_vs_t(tp=TTp,phip=PHIP,tm=TTm,phim=PHIM,color=color,mylabel=mylabel,phi=PHI)
            elif plotme=='z vs t':
                both_z_vs_t(tp=TTp,zp=ZP,tm=TTm,zm=ZM,color=color,mylabel=mylabel,zc=zc_act)
plt.legend(loc='upper right', prop={'size': 14})
plt.show()

Using matplotlib backend: Qt5Agg
Processing detector 1
z_C : 1.302 m
30987 30766
Both images moving. Processing...
Processing detector 2
z_C : 1.471 m
30064 30911
Both images moving. Processing...
Processing detector 3
z_C : 3.623 m
11055 30911
Both images moving. Processing...
Processing detector 4
z_C : 2.743 m
20407 30911
Both images moving. Processing...
